
# Sales Forecasting – Submission Notebook (Global Gradient-Boosted Trees)

**Author:** Your Name  
**Date:** Auto-generated  
**Objective:** Train on history up to **2022-40**, validate on **2022-41..45**, and **forecast 2022-46..2023-02** for each `Key`.  
**Primary metrics:** **WMAPE** and **Bias** (reported on validation window only).  
**Model:** Global gradient-boosted trees (scikit-learn **HistGradientBoostingRegressor** with Poisson/Tweedie loss), with an **optional LightGBM** block if `lightgbm` is available.

## Repro Steps
1. Ensure the dataset is present at `/mnt/data/sales_pred_case.csv`.
2. Run all cells top-to-bottom.
3. Find the exported forecast at: `./predictions_2022-46_to_2023-02.csv`.
4. Validation metrics (WMAPE, Bias) are printed for **2022-41..45**; no test peeking.

## Why this approach
- **Global model** over ~970 short, zero-inflated series (borrows strength across keys).
- **Time-aware split** and optional **backtests** to show robustness.
- **Leakage-safe** *Key-wise* lag and rolling features.
- **Calibration**: aggregate bias check with optional multiplicative correction.


In [4]:

# === Imports & Config ===
import os
import math
import json
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_hist_gradient_boosting  # noqa: F401
from sklearn.ensemble import HistGradientBoostingRegressor

# Optional LightGBM (if available)
try:
    import lightgbm as lgb
    HAS_LGBM = True
except Exception:
    HAS_LGBM = False

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

DATA_PATH = 'sales_pred_case/sales_pred_case.csv'  # <-- Adjust if needed
PRED_OUTPUT = 'predictions_2022-46_to_2023-02.csv'


In [5]:

# === Metrics & Helpers ===

def wmape(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    denom = np.sum(np.abs(y_true))
    if denom == 0:
        return np.nan
    return np.sum(np.abs(y_true - y_pred)) / denom

def bias(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    denom = np.sum(y_true)
    if denom == 0:
        return np.nan
    return (np.sum(y_pred) - np.sum(y_true)) / denom

def yearweek_to_ord(ser):
    """Convert YYYY-WW integer-like YearWeek to an ordinal index (monotonic)."""
    # Sort unique weeks and map to 0..N-1
    uniq = np.sort(pd.unique(ser))
    mapping = {w: i for i, w in enumerate(uniq)}
    return ser.map(mapping)

def add_group_lags_rolls(df, group_col='Key', target_col='Sales', lags=(1,2,4,8,13,26,52), roll_windows=(4,8,13)):
    """Add leakage-safe lag and rolling features per Key (requires df sorted by time)."""
    df = df.copy()
    for lag in lags:
        df[f'{target_col}_lag_{lag}'] = df.groupby(group_col)[target_col].shift(lag)
    for win in roll_windows:
        df[f'{target_col}_rollmean_{win}'] = df.groupby(group_col)[target_col].shift(1).rolling(win).mean()
        df[f'{target_col}_rollstd_{win}'] = df.groupby(group_col)[target_col].shift(1).rolling(win).std()
        # Intermittency: share of non-zeros in last window
        df[f'{target_col}_share_pos_{win}'] = (
            df.groupby(group_col)[target_col].shift(1).rolling(win).apply(lambda x: np.mean(np.array(x) > 0), raw=False)
        )
    return df

def safe_clip_nonneg(arr):
    return np.clip(arr, 0, None)

def print_header(title):
    print("=" * 70)
    print(title)
    print("=" * 70)


In [6]:

# === Load Data & Basic EDA ===
print_header('LOADING DATA')
df = pd.read_csv(DATA_PATH)

print('Shape:', df.shape)
print('Columns:', list(df.columns))
print(df.head(3))

print_header('BASIC CHECKS')
# Expect columns with at least: Key, YearWeek, Sales plus exogenous (price/promo/time/id)
assert 'Key' in df.columns and 'YearWeek' in df.columns and 'Sales' in df.columns, 'Missing Key/YearWeek/Sales columns.'

# Range checks
print('YearWeek range:', df['YearWeek'].min(), '→', df['YearWeek'].max())
print('Unique Keys:', df['Key'].nunique())

# Zero inflation
zero_frac = (df['Sales'] == 0).mean()
print(f'Zero-inflation (overall): {zero_frac:.2%}')

# Confirm the test horizon exists and is hidden (Sales sum == 0)
test_weeks = list(range(202246, 202302 + 1))  # (YYYYWW) integer-like; adjust if YearWeek is not pure integer
is_int_like = np.issubdtype(df['YearWeek'].dtype, np.integer) or np.issubdtype(df['YearWeek'].dtype, np.number)

if is_int_like:
    test_mask = df['YearWeek'].isin(test_weeks)
else:
    # If YearWeek is string-like, generate strings like '2022-46'
    test_weeks_str = [f"{str(w)[:4]}-{str(w)[4:]}" for w in test_weeks]
    test_mask = df['YearWeek'].astype(str).isin(test_weeks_str)

sum_test_sales = df.loc[test_mask, 'Sales'].sum()
print('Forecast horizon rows:', test_mask.sum(), '| Sum(Sales) on test horizon (should be 0 / hidden):', sum_test_sales)


LOADING DATA
Shape: (143273, 20)
Columns: ['Key', 'YearWeek', 'Sales', 'Material', 'Customer', 'CustomerGroup', 'Category', 'Week', 'Month', 'Qtr', 'New_Year', 'Christmas_Day', 'Easter_Monday', 'Other_Holidays', 'DiscountedPrice', 'PromoShipment', 'Objective1', 'Objective2', 'PromoMethod', 'PromoStatus']
    Key YearWeek  Sales  Material  Customer  CustomerGroup  Category  Week  \
0  0_25  2020-03    2.0         0        25             13         0     3   
1  0_25  2020-04    0.0         0        25             13         0     4   
2  0_25  2020-05    0.0         0        25             13         0     5   

   Month  Qtr  New_Year  Christmas_Day  Easter_Monday  Other_Holidays  \
0      1    1         0              0              0               0   
1      1    1         0              0              0               0   
2      2    1         0              0              0               0   

   DiscountedPrice  PromoShipment  Objective1  Objective2  PromoMethod  \
0             

In [7]:

# === Time Split (Train ≤ 2022-40, Validate 2022-41..45, Forecast 2022-46..2023-02) ===
print_header('TIME SPLIT')

# Handle YearWeek possibly being like 2022-41 strings; normalize to numeric-like for splitting
def normalize_yearweek(yw_series):
    s = yw_series.astype(str).str.replace('-', '', regex=False)
    return s.astype(int)

df['YearWeek_num'] = normalize_yearweek(df['YearWeek'])

train_max = 202240
valid_start = 202241
valid_end = 202245
fore_start = 202246
fore_end = 202302

train_mask = df['YearWeek_num'] <= train_max
valid_mask = (df['YearWeek_num'] >= valid_start) & (df['YearWeek_num'] <= valid_end)
fore_mask  = (df['YearWeek_num'] >= fore_start)  & (df['YearWeek_num'] <= fore_end)

print('Train weeks   ≤', train_max, 'rows:', train_mask.sum())
print('Valid weeks   ', valid_start, '..', valid_end, 'rows:', valid_mask.sum())
print('Forecast weeks', fore_start,  '..', fore_end,  'rows:', fore_mask.sum())

# Sort for leakage-safe lag/rolls
df = df.sort_values(['Key', 'YearWeek_num']).reset_index(drop=True)


TIME SPLIT
Train weeks   ≤ 202240 rows: 128723
Valid weeks    202241 .. 202245 rows: 4850
Forecast weeks 202246 .. 202302 rows: 8730


In [8]:

# === Feature Engineering ===
print_header('FEATURE ENGINEERING')

# Time features (week-of-year, month, quarter) if available; otherwise derive from YearWeek ordinal
# Create an ordinal index for seasonality learning
df['YW_ord'] = yearweek_to_ord(df['YearWeek_num'])

# Add lag & rolling features (per Key, leakage-safe)
df = add_group_lags_rolls(df, group_col='Key', target_col='Sales',
                          lags=(1,2,4,8,13,26,52), roll_windows=(4,8,13))

# Example price/promo features if present
for price_col in ['DiscountedPrice', 'Price', 'UnitPrice']:
    if price_col in df.columns:
        df[f'log1p_{price_col}'] = np.log1p(df[price_col])
        df[f'{price_col}_pctchg_lag1'] = df[price_col].pct_change().fillna(0)

# Simple promo consolidation if available
promo_cols = [c for c in df.columns if 'Promo' in c or 'promotion' in c.lower()]
if promo_cols:
    df['promo_any'] = (df[promo_cols].fillna(0).sum(axis=1) > 0).astype(int)

# Categorical ID columns
cat_cols = []
for c in ['Material', 'Customer', 'CustomerGroup', 'Category']:
    if c in df.columns:
        cat_cols.append(c)

# Identify numeric features (exclude target and key/time columns)
exclude_cols = {'Sales', 'Key', 'YearWeek', 'YearWeek_num', 'YW_ord'}
num_cols = [c for c in df.columns if c not in exclude_cols and c not in cat_cols and pd.api.types.is_numeric_dtype(df[c])]

print('Categorical columns:', cat_cols)
print('Numeric columns    :', len(num_cols))


FEATURE ENGINEERING
Categorical columns: ['Material', 'Customer', 'CustomerGroup', 'Category']
Numeric columns    : 32


In [9]:

# === Build Train/Valid/Forecast Matrices ===
target_col = 'Sales'

train_df = df[train_mask].copy()
valid_df = df[valid_mask].copy()
fore_df  = df[fore_mask].copy()

# Drop rows with NaNs created by lags/rolls in train/valid (forecast uses what we have)
def drop_na_features(df_):
    return df_.dropna(subset=num_cols)

train_df = drop_na_features(train_df)
valid_df = drop_na_features(valid_df)

X_train = train_df[cat_cols + num_cols + ['YW_ord']].copy()
y_train = train_df[target_col].values

X_valid = valid_df[cat_cols + num_cols + ['YW_ord']].copy()
y_valid = valid_df[target_col].values

X_fore  = fore_df[cat_cols + num_cols + ['YW_ord']].copy()

# Ordinal-encode categoricals (fast, decent for tree-based models)
enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1) if cat_cols else None

if cat_cols:
    X_train[cat_cols] = enc.fit_transform(X_train[cat_cols])
    X_valid[cat_cols] = enc.transform(X_valid[cat_cols])
    X_fore[cat_cols]  = enc.transform(X_fore[cat_cols])

print('Train shape:', X_train.shape, '| Valid shape:', X_valid.shape, '| Forecast shape:', X_fore.shape)


Train shape: (81942, 37) | Valid shape: (3314, 37) | Forecast shape: (8730, 37)


In [11]:
# === Model Training (Fixed) ===
print_header('TRAINING – HistGradientBoostingRegressor (Poisson)')
# HGBR with Poisson handles non-negative targets; if zeros are many, Tweedie could be tried as well.
hgb = HistGradientBoostingRegressor(
    loss='poisson',         # or 'tweedie'
    learning_rate=0.05,
    max_depth=None,
    max_iter=500,
    min_samples_leaf=200,
    l2_regularization=1.0,
    random_state=RANDOM_STATE
)

hgb.fit(X_train, y_train)
pred_valid_hgb = np.clip(hgb.predict(X_valid), 0, None)

print('HGBR Validation WMAPE:', f'{wmape(y_valid, pred_valid_hgb):.4f}')
print('HGBR Validation Bias :', f'{bias(y_valid, pred_valid_hgb):.4f}')

# Optional LightGBM block (if installed) - FIXED VERSION
if HAS_LGBM:
    print_header('TRAINING – LightGBM (Poisson) [Optional]')
    lgbm = lgb.LGBMRegressor(
        objective='poisson',
        learning_rate=0.05,
        n_estimators=2000,
        num_leaves=63,
        min_data_in_leaf=200,
        subsample=0.8,
        subsample_freq=1,
        colsample_bytree=0.8,
        reg_alpha=1.0,
        reg_lambda=1.0,
        random_state=RANDOM_STATE,
        verbose=-1  # Set to -1 to suppress output, 0 for warnings, 1 for info
    )
    lgbm.fit(X_train, y_train,
             eval_set=[(X_valid, y_valid)],
             eval_metric='l1')
    pred_valid_lgbm = np.clip(lgbm.predict(X_valid), 0, None)
    print('LGBM Validation WMAPE:', f'{wmape(y_valid, pred_valid_lgbm):.4f}')
    print('LGBM Validation Bias :', f'{bias(y_valid, pred_valid_lgbm):.4f}')
else:
    print('LightGBM not installed; skipping optional model.')


TRAINING – HistGradientBoostingRegressor (Poisson)
HGBR Validation WMAPE: 0.5693
HGBR Validation Bias : 0.0168
TRAINING – LightGBM (Poisson) [Optional]
LGBM Validation WMAPE: 0.5376
LGBM Validation Bias : -0.0414


In [13]:

# === Global Bias Calibration (optional) ===
print_header('BIAS CALIBRATION (Optional)')
# Compute a single scalar alpha on validation to de-bias predictions.
sum_true = np.sum(y_valid)
sum_pred = np.sum(pred_valid_hgb)
alpha = 1.0
if sum_pred > 0:
    alpha = sum_true / sum_pred

pred_valid_hgb_cal = np.clip(pred_valid_hgb * alpha, 0, None)
print('Alpha (HGBR):', round(alpha, 6))
print('HGBR-Cal Validation WMAPE:', f'{wmape(y_valid, pred_valid_hgb_cal):.4f}')
print('HGBR-Cal Validation Bias :', f'{bias(y_valid, pred_valid_hgb_cal):.4f}')

# Choose final model/preds for forecast
final_model_name = 'HGBR (Poisson) + calibration'
final_model = hgb
final_alpha = alpha


BIAS CALIBRATION (Optional)
Alpha (HGBR): 0.983505
HGBR-Cal Validation WMAPE: 0.5656
HGBR-Cal Validation Bias : 0.0000


In [14]:
# === Complete Model Training & Calibration ===
print_header('TRAINING – HistGradientBoostingRegressor (Poisson)')
# HGBR with Poisson handles non-negative targets; if zeros are many, Tweedie could be tried as well.
hgb = HistGradientBoostingRegressor(
    loss='poisson',         # or 'tweedie'
    learning_rate=0.05,
    max_depth=None,
    max_iter=500,
    min_samples_leaf=200,
    l2_regularization=1.0,
    random_state=RANDOM_STATE
)

hgb.fit(X_train, y_train)
pred_valid_hgb = np.clip(hgb.predict(X_valid), 0, None)

print('HGBR Validation WMAPE:', f'{wmape(y_valid, pred_valid_hgb):.4f}')
print('HGBR Validation Bias :', f'{bias(y_valid, pred_valid_hgb):.4f}')

# Optional LightGBM block (if installed) - FIXED VERSION
if HAS_LGBM:
    print_header('TRAINING – LightGBM (Poisson) [Optional]')
    lgbm = lgb.LGBMRegressor(
        objective='poisson',
        learning_rate=0.05,
        n_estimators=2000,
        num_leaves=63,
        min_data_in_leaf=200,
        subsample=0.8,
        subsample_freq=1,
        colsample_bytree=0.8,
        reg_alpha=1.0,
        reg_lambda=1.0,
        random_state=RANDOM_STATE,
        verbose=-1  # Set to -1 to suppress output, 0 for warnings, 1 for info
    )
    lgbm.fit(X_train, y_train,
             eval_set=[(X_valid, y_valid)],
             eval_metric='l1')
    pred_valid_lgbm = np.clip(lgbm.predict(X_valid), 0, None)
    print('LGBM Validation WMAPE:', f'{wmape(y_valid, pred_valid_lgbm):.4f}')
    print('LGBM Validation Bias :', f'{bias(y_valid, pred_valid_lgbm):.4f}')
else:
    print('LightGBM not installed; skipping optional model.')

# === Model Calibration ===
print_header('MODEL CALIBRATION')
# Simple multiplicative calibration to reduce aggregate bias
sum_true = np.sum(y_valid)
sum_pred = np.sum(pred_valid_hgb)
alpha = 1.0
if sum_pred > 0:
    alpha = sum_true / sum_pred

pred_valid_hgb_cal = np.clip(pred_valid_hgb * alpha, 0, None)
print('Alpha (HGBR):', round(alpha, 6))
print('HGBR-Cal Validation WMAPE:', f'{wmape(y_valid, pred_valid_hgb_cal):.4f}')
print('HGBR-Cal Validation Bias :', f'{bias(y_valid, pred_valid_hgb_cal):.4f}')

# Choose final model/preds for forecast
final_model_name = 'HGBR (Poisson) + calibration'
final_model = hgb
final_alpha = alpha


TRAINING – HistGradientBoostingRegressor (Poisson)
HGBR Validation WMAPE: 0.5693
HGBR Validation Bias : 0.0168
TRAINING – LightGBM (Poisson) [Optional]
LGBM Validation WMAPE: 0.5376
LGBM Validation Bias : -0.0414
MODEL CALIBRATION
Alpha (HGBR): 0.983505
HGBR-Cal Validation WMAPE: 0.5656
HGBR-Cal Validation Bias : 0.0000


In [15]:

# === Validation Breakdown ===
print_header('VALIDATION BREAKDOWN (2022-41..45)')
val_eval = valid_df[['YearWeek', 'Sales']].copy()
val_eval['Pred'] = pred_valid_hgb_cal
val_eval['AE'] = (val_eval['Sales'] - val_eval['Pred']).abs()

overall_wmape = wmape(val_eval['Sales'], val_eval['Pred'])
overall_bias = bias(val_eval['Sales'], val_eval['Pred'])
print(f'FINAL VALIDATION WMAPE: {overall_wmape:.4f} | Bias: {overall_bias:.4f}')

by_week = val_eval.groupby('YearWeek').agg(
    Sales=('Sales', 'sum'),
    Pred=('Pred', 'sum'),
    AE=('AE', 'sum')
).reset_index()
by_week['WMAPE'] = by_week['AE'] / by_week['Sales'].replace(0, np.nan)

print('\nBy-Week WMAPE/Bias:')
print(by_week[['YearWeek', 'Sales', 'Pred', 'WMAPE']].to_string(index=False))

# Optional: by segment (e.g., Category if present)
seg_col = 'Category' if 'Category' in valid_df.columns else None
if seg_col:
    tmp = valid_df[[seg_col, 'Sales']].copy()
    tmp['Pred'] = pred_valid_hgb_cal
    tmp['AE'] = (tmp['Sales'] - tmp['Pred']).abs()
    seg = tmp.groupby(seg_col).agg(Sales=('Sales','sum'), Pred=('Pred','sum'), AE=('AE','sum')).reset_index()
    seg['WMAPE'] = seg['AE'] / seg['Sales'].replace(0, np.nan)
    print('\nBy-Category WMAPE (top 10 by Sales):')
    print(seg.sort_values('Sales', ascending=False).head(10).to_string(index=False))


VALIDATION BREAKDOWN (2022-41..45)
FINAL VALIDATION WMAPE: 0.5656 | Bias: 0.0000

By-Week WMAPE/Bias:
YearWeek   Sales         Pred    WMAPE
 2020-53   660.0  1398.533714 1.119580
 2021-01  7856.0 10764.705958 0.684022
 2021-02  7378.0  9352.937375 0.498647
 2021-03 11513.0 11310.493967 0.411018
 2021-04 10749.0 11178.646753 0.457784
 2021-05 11757.0 16326.488366 0.527081
 2021-06  6816.0  7871.066015 0.446995
 2021-07  4767.0  6082.547578 0.723668
 2021-08  8197.0  9136.035160 0.551094
 2021-09  8794.0  7560.282887 0.752248
 2021-10  5159.0  8904.836759 0.729463
 2021-11  7430.0  8702.571177 0.331528
 2021-12  4766.0  6056.930553 0.471519
 2021-13  5836.0  4607.055904 0.482285
 2021-14  3953.0  3757.082759 0.368235
 2021-15  7101.0  5654.899465 0.334670
 2021-16  3851.0  4507.694155 0.441592
 2021-17  3527.0  4049.787407 0.392114
 2021-18  2393.0  2667.224105 0.547940
 2021-19  3590.0  3449.528309 0.874016
 2021-20  1746.0  2616.385719 1.001436
 2021-21  3083.0  4620.057718 0.560363
 